In [ ]:
import pandas as pd

Import the dataset as a DataFrame and storage it in the data variable

In [ ]:
data = pd.read_csv('_src/data/movies_dataset.csv', encoding='UTF-8', delimiter=',', )

Import the credits dataset for later on extract the director name

In [ ]:
credits = pd.read_csv('_src/data/credits.csv')
credits.drop(columns=['cast'], inplace=True)

Transforme id into a integer

In [ ]:
try:
    data['id'].astype(int)
except ValueError as e:
    print(e)

We can see that the data is all wrong, so we drop it

In [ ]:
data.drop(data.loc[data['id'].str.contains('-', regex=True)].index, inplace=True)

In [ ]:
data['id'] = data['id'].astype(int)

Merge the dataframes so we work with only 1

In [ ]:
data = pd.merge(data,credits, on='id')

Drop the innecesary columns

In [ ]:
data.drop(columns=['adult','homepage','imdb_id','original_title','poster_path','video','spoken_languages'], inplace = True)

### Extract the nested data that we need and create its dataframe's columns ### 

Nested data in 'belongs_to_collection'

In [ ]:
#Crate the new columns of the DataFrame
data['collection_name'] = 0

#Asign to the new columns the values in belongs_to_collecition < id and name
for i in range(len(data['belongs_to_collection'])):
    if (len(str(data['belongs_to_collection'][i])) > 10):
        data['belongs_to_collection'][i] = data['belongs_to_collection'][i].split(',')
        data.loc[i, 'collection_name'] = data['belongs_to_collection'][i][1][10:-1]

#Drop belongs_to_collection
data.drop(columns = 'belongs_to_collection', inplace = True)

nested data in 'crew'

In [ ]:
def extract_directors(crew_str):
    member_dict = ast.literal_eval(crew_str)
    directors = [member_dict[i]['name'] for i in range(len(member_dict)) if member_dict[i]['job'] == 'Director']
    return ','.join(directors)

data['directors'] = data['crew'].apply(extract_directors)

Nested data in 'genres'

In [ ]:
# Define a function to extract genre names from genres
def extract_genre_names(genres_str):
    if(len(str(genres_str)) > 4):
        genre_list = genres_str.split(',')
        genre_names = [element.strip(']')[10:-2] for element in genre_list if element[2:6] == 'name']
        return ','.join(genre_names)

# Apply the function to create the 'genres_list' column
data['genres_list'] = data['genres'].apply(extract_genre_names)

# Drop 'genres' column
data.drop(columns='genres', inplace=True)

Nested data in 'production_companies'

In [ ]:
#Crate the new columns of the DataFrame
data['prod_companies'] = ''

def extract_comp_names(comp_str):
    if(len(str(comp_str)) > 4):
        genre_list = comp_str.split(',')
        comp_names = [element.strip(']')[11:-1] for element in genre_list if element[3:7] == 'name']
        return ','.join(comp_names)

# Apply the function to create the 'genres_list' column
data['prod_companies'] = data['production_companies'].apply(extract_comp_names)

# Drop 'genres' column
data.drop(columns='production_companies', inplace = True)

data['prod_companies'].fillna('None', inplace = True)

Nested data in 'production_countries'

In [ ]:
def extract_countries_names(country_str):
    if(len(str(country_str)) > 4):
        # countries_list = country_str.str.strip('([])')
        countries_list = country_str.split(',')
        countries_names = [element.strip("]")[10:-2] for element in countries_list if element[2:6] == 'name']
        return ','.join(countries_names)

data['prod_countries'] = data['production_countries'].apply(extract_countries_names)

data.drop(columns = 'production_countries')

Nested data in 'original_language'

In [ ]:
# data['release_language'] = data['spoken_languages'].apply(lambda x: x.split(',')[1][10:-2].strip("'") if len(str(x)) > 4 else '')

# data.drop(columns='spoken_languages', inplace=True)

In [ ]:
data.head(7)

Create a column 'return' with the return of investmen. (revenue / budget). In case that there is no data to operate it has to take the value '0' 

Fill the nulls values in budget and revenue with 0

In [ ]:
fill_values = {'revenue': 0, 'budget': 0}
data.fillna(fill_values, inplace=True)

Now that we have all the data, we change the dtypes of the necesary columns to make the operations

In [ ]:
int_columns = ['budget','revenue']
float_columns = ['popularity','vote_average']

data[int_columns] = data[int_columns].astype(int)
data[float_columns] = data[float_columns].astype(float)

Create the 'return' column

In [ ]:
data['return'] = round(data['revenue'] / data['budget'],2)

In [ ]:
data.loc[data['return'] == float('inf'), 'return'] = data['revenue']

The dates has to have the format AAAA-mm-dd, and with that create the column release_year where we will extract the year of release

In [ ]:
data['release_date'] = pd.to_datetime(data['release_date'])
data.dropna(subset=['release_date'], inplace=True)
data['release_year'] = data['release_date'].dt.year

In [ ]:
new_columns = ['collection_name','directors', 'genres_list', 'id', 
               'title', 'prod_companies', 'prod_countries', 'original_language', 
               'release_language', 'overview', 'popularity', 'release_date','release_year', 'runtime', 
               'status', 'tagline', 'vote_average', 'vote_count', 'budget', 'revenue','return']
data = data.reindex(columns = new_columns)

In [ ]:
data.loc[data['prod_companies'].isna(), 'title']

Drop the movies with no release_date

In [ ]:
data.drop(data.loc[data['release_date'].isna()].index, inplace=True)

In [ ]:
data.to_csv('_src/data/movies_transformed.csv', index=False)